In [ ]:
import numpy as np
import pandas as pd
import copy
import scipy.special as spsp
import scipy.stats as spst
import matplotlib.pyplot as plt
%matplotlib inline
import sympy
!pip install simpy
import simpy
 

2 types of arrival

unit of time: hours

Online lambda = 240

Offline lambda = 60

assume online & offline independent. The total arrival process N(t) = N1 + N2

number of workers =10, indifferent

types of drinks = 5 with different preparing time

the limit for online and offline queue is 150 and 60 (could be changed)

## STRATEGY1: FIFO

### System

In [ ]:
def select_product(u):
  if u <= drinks[1]['prob']:
    return 1
  elif u <= drinks[1]['prob']+drinks[2]['prob']:
    return 2
  elif u <= drinks[1]['prob']+drinks[2]['prob']+drinks[3]['prob']:
    return 3
  elif u <= drinks[1]['prob']+drinks[2]['prob']+drinks[3]['prob']+drinks[4]['prob']:
    return 4
  else:
    return 5


def arrival_fifo(env,worker,online_lm,offline_lm,customers,outputs,wait_prepare,T,RS_s,RS_a,anti):
  i = 0 
  lmbda = online_lm + offline_lm
  while env.now<T: 
    u = RS_a.rand()
    if anti==1:
      u = 1-u
    inter_arrival =-1/lmbda*np.log(u)
    yield env.timeout(inter_arrival)
    i+=1
    if np.random.rand() < online_lm/(lmbda): # if it is a online order
        customers[i]['channel'] = 'online'
        customers[i]['order_amount'] = int(np.log(np.random.rand())/np.log(1-multi_order_prob_online))+1
        customers[i]['arrival'] = env.now
        for j in range(customers[i]['order_amount']):
          customers[i]['orders'].append(select_product(np.random.rand()))
        if wait_prepare['online'] <= online_limit:
          wait_prepare['online']+=1
          env.process(service_fifo(env,worker,i,customers,outputs,wait_prepare,RS_s,anti)) 
        else: # we lost this customer, record the potential revenue
          #print(f"we lost customer {i} who is a {customers[i]['channel']} customer,because we have {wait_prepare['online']} orders waiting")
          outputs['lost_online_customer'] += 1
          for drink in customers[i]['orders']:    
            outputs['lost_online_revenue'] += drinks[drink]['price']*0.9                                 
      # record this online order
    else: # it is a offline order
      customers[i]['channel'] = 'offline'
      customers[i]['order_amount'] = int(np.log(np.random.rand())/np.log(1-multi_order_prob_offline))+1
      customers[i]['arrival'] = env.now
      for j in range(customers[i]['order_amount']):
        customers[i]['orders'].append(select_product(np.random.rand()))
      if wait_prepare['offline'] <= offline_limit: # 
        wait_prepare['offline']+=1
        env.process(service_fifo(env,worker,i,customers,outputs,wait_prepare,RS_s,anti))
      else:
          #print(f"we lost customer {i} who is a {customers[i]['channel']} customer, because we have {wait_prepare['offline']} orders waiting")
          outputs['lost_offline_customer'] += 1
          for drink in customers[i]['orders']:    
            outputs['lost_offline_revenue'] += drinks[drink]['price']*0.9      


def service_fifo(env,worker,i,customers,outputs,wait_prepare,RS_s,anti):
  order_channel = customers[i]['channel']
  #print(f"At {env.now:.2f} customer {i} makes order, it is a {customers[i]['channel']} order with quantity = {customers[i]['order_amount']} and drink types are {customers[i]['orders']}")
  for drink in customers[i]['orders']:
    lm = drinks[drink]['rate']
    if order_channel == 'online':
      lm += online_rate_premium
    u = RS_s.rand()
    if anti == 1:
      u = 1-u
    preparing_time = -1/lm*np.log(u)
    # if customers[i]['channel']=='offline':
    rqt = worker.request(priority = 0)
    yield rqt
    yield env.timeout(preparing_time)
    worker.release(rqt)
    if customers[i]['channel']=='offline':
      customers[i]['revenue']+= drinks[drink]['price']
    else: # discount provided to online order/ or the platform(Meituan) need to have a distribution fee.
      customers[i]['revenue']+= drinks[drink]['price'] * online_discount
  if customers[i]['channel']=='offline':  
    outputs['offline_total'] += 1
  else:
    outputs['online_total'] += 1
  customers[i]['finish'] = env.now
  customers[i]['waiting'] = customers[i]['finish']-customers[i]['arrival']
  # linear wating time penalty
  customers[i]['penalized_revenue'] = customers[i]['revenue']*(1-waiting_penalty*customers[i]['waiting'])
  # exponential waiting time penalty
  #customers[i]['penalized_revenue'] = customers[i]['revenue']*np.exp(-waiting_penalty*customers[i]['waiting'])
  if order_channel == 'offline':
    wait_prepare['offline'] -=1
  else:
    wait_prepare['online'] -=1
  #print(f"customer {i} order finished at {env.now:.2f}")


def system_fifo(RS_s,RS_a,anti):
  env = simpy.Environment()
  customers = {i:{"channel":"online","order_amount":0,"orders":[],"arrival":0,"finish" :0,"revenue":0,'waiting':0,'penalized_revenue':0} for i in range(1,10000)}
  outputs = {'online_total':0,'offline_total':0,'lost_online_customer':0,'lost_offline_customer':0,'lost_online_revenue':0,'lost_offline_revenue':0}
  wait_prepare = {"online":0,"offline":0}
  T = 4  # after 11 hours, stop accept new orders
  worker = simpy.PriorityResource(env,capacity=10)
  env.process(arrival_fifo(env,worker,online_arr_rate,offline_arr_rate,customers,outputs,wait_prepare,T,RS_s,RS_a,anti))
  env.run()
  total_revenue = sum([customers[i]['revenue'] for i in range(1,10000)])
  total_penlized_revenue = sum([customers[i]['penalized_revenue'] for i in range(1,10000)])
  total_orders = outputs['online_total']+outputs['offline_total']
  average_waiting = sum([customers[i]['waiting'] for i in range(1,10000)])/total_orders
  total_lost_orders = outputs['lost_online_customer'] + outputs['lost_offline_customer']
  total_lost_revenue = outputs['lost_online_revenue']+outputs['lost_offline_revenue']
  #print(f"total revenue is {total_revenue:.2f}")
  return [total_revenue,total_penlized_revenue,average_waiting,total_orders,total_lost_orders,total_lost_revenue]


def system_fifo_avg(s=0):
  np.random.seed(s)
  RS_s_original=np.random.RandomState(seed=s)
  RS_s_anti=copy.copy(RS_s_original)
  RS_a_original=np.random.RandomState(seed=1000-s)
  RS_a_anti=copy.copy(RS_a_original)
  output1 = system_fifo(RS_s_original,RS_a_original,0)
  output2 = system_fifo(RS_s_anti,RS_a_anti,1)
  return [(g+h)/2 for g,h in zip(output1,output2)]


def get_CI(array,name):
  N=len(array)
  mean=np.mean(array)
  std=np.std(array,ddof=1)
  lower=mean-spst.norm.ppf(0.975)*std/N**0.5
  upper=mean+spst.norm.ppf(0.975)*std/N**0.5
  print("["+name+"] Mean: %.3f, Std: %.3f, 95%% CI: [%.3f, %.3f], Width: %.3f, N: %d"%(mean,std,lower,upper,upper-lower,N))

### Parameter

In [ ]:
N = 100 # number of simulation rounds for Strategy1: FIFO
N_search = 100 # number of simulation rounds for Strategy2: Priority Balance

# Different drinks sold offline
drinks =  {1:{"price":22,"prob":0.3,"rate":32},
        2:{"price":25,"prob":0.25,"rate":30},
        3:{"price":28,"prob":0.2,"rate":28},
        4:{"price":30,"prob":0.15,"rate":25},
        5:{"price":32,"prob":0.1,"rate":22}}

multi_order_prob_offline = 0.9 # geometric p for multi-ordering for offline
multi_order_prob_online = 0.7 # geometric p for multi-ordering for online
online_limit = 200 
offline_limit = 30
online_arr_rate = 240 # online arrival rate
offline_arr_rate = 120
online_discount = 0.65 # online price discount rate
online_rate_premium = 1 # working rate is higher for online orders
waiting_penalty = 0.2 # penalty for waiting time

### Simulation

In [ ]:
# store
result_fifo = [system_fifo_avg(i) for i in range(N)]
result_fifo = np.array(result_fifo)
total_revenue_fifo = result_fifo[:,0]
total_penlized_revenue_fifo = result_fifo[:,1]
average_waiting_fifo =  result_fifo[:,2]
total_orders_fifo = result_fifo[:,3]
total_lost_orders_fifo = result_fifo[:,4]
total_lost_revenue_fifo = result_fifo[:,5]

### Performance

In [ ]:
# print
get_CI(total_revenue_fifo,"total_revenue_fifo")
get_CI(total_penlized_revenue_fifo,"total_penlized_revenue_fifo")
get_CI(average_waiting_fifo,"average_waiting_fifo")
get_CI(total_orders_fifo,"total_orders_fifo")
get_CI(total_lost_orders_fifo,"total_lost_orders_fifo")
get_CI(total_lost_revenue_fifo,"total_lost_revenue_fifo")

[total_revenue_fifo] Mean: 27437.976, Std: 303.641, 95% CI: [27378.463, 27497.488], Width: 119.025, N: 100
[total_penlized_revenue_fifo] Mean: 22304.576, Std: 378.075, 95% CI: [22230.475, 22378.677], Width: 148.203, N: 100
[average_waiting_fifo] Mean: 0.757, Std: 0.025, 95% CI: [0.752, 0.762], Width: 0.010, N: 100
[total_orders_fifo] Mean: 1082.270, Std: 16.587, 95% CI: [1079.019, 1085.521], Width: 6.502, N: 100
[total_lost_orders_fifo] Mean: 356.755, Std: 23.215, 95% CI: [352.205, 361.305], Width: 9.100, N: 100
[total_lost_revenue_fifo] Mean: 10146.479, Std: 761.495, 95% CI: [9997.228, 10295.729], Width: 298.501, N: 100


# STRATEGY2: Priority Balance

### System

Specify a theta which equals to n1(online)/n2(offline)

The service will provide sequantially to online and offline order with a ratio of theta. For example, if theta = 1, then we need to have make a offline order after the online order, and the next offline order has to wait

We trace the number of online order and offline order, say, if there are all online orders, then we keep the FIFO process, but if there is a online order coming in and can not satisfy the theta constraint, we put priority to the offline order.

More specifically, because the online order amount is far larger than the offline order, but the offline order is actually provide more revenue. So we would like to use theta to give some priority to offline orders.

In [ ]:
def arrival_prio(env,worker,online_lm,offline_lm,customers,outputs,wait_prepare,T,theta,RS_s,RS_a,anti):
  i = 0 
  lmbda = online_lm + offline_lm
  while env.now<T: 
    u = RS_a.rand()
    if anti==1:
      u = 1-u
    inter_arrival =-1/lmbda*np.log(u)
    yield env.timeout(inter_arrival)
    i+=1
    if np.random.rand() < online_lm/(lmbda): # if it is a online order
        customers[i]['channel'] = 'online'
        customers[i]['order_amount'] = int(np.log(np.random.rand())/np.log(1-multi_order_prob_online))+1
        customers[i]['arrival'] = env.now
        for j in range(customers[i]['order_amount']):
          customers[i]['orders'].append(select_product(np.random.rand()))
        if wait_prepare['online'] <= online_limit:
          wait_prepare['online']+=1
          env.process(service_prio(env,worker,i,customers,outputs,wait_prepare,theta,RS_s,anti)) 
        else: # we lost this customer, record the potential revenue
          #print(f"we lost customer {i} who is a {customers[i]['channel']} customer,because we have {wait_prepare['online']} orders waiting")
          outputs['lost_online_customer'] += 1
          for drink in customers[i]['orders']:    
            outputs['lost_online_revenue'] += drinks[drink]['price']*0.9                                 
      # record this online order
    else: # it is a offline order
      customers[i]['channel'] = 'offline'
      customers[i]['order_amount'] = int(np.log(np.random.rand())/np.log(1-multi_order_prob_offline))+1
      customers[i]['arrival'] = env.now
      for j in range(customers[i]['order_amount']):
        customers[i]['orders'].append(select_product(np.random.rand()))
      if wait_prepare['offline'] <= offline_limit: # 
        wait_prepare['offline']+=1
        env.process(service_prio(env,worker,i,customers,outputs,wait_prepare,theta,RS_s,anti))
      else:
          #print(f"we lost customer {i} who is a {customers[i]['channel']} customer, because we have {wait_prepare['offline']} orders waiting")
          outputs['lost_offline_customer'] += 1
          for drink in customers[i]['orders']:    
            outputs['lost_offline_revenue'] += drinks[drink]['price']*0.9      
      
def service_prio(env,worker,i,customers,outputs,wait_prepare,theta,RS_s,anti):
  order_channel = customers[i]['channel']
  current_ratio = outputs['online_total']/(outputs['offline_total']+0.000000001)
  # print("current ratio is:",current_ratio)
  if (current_ratio >= theta) and (order_channel=='offline'):   
    priority = -1
  else:
    priority = 0
  #print(f"At {env.now:.2f} customer {i} makes order, it is a {priority} {customers[i]['channel']} order with quantity = {customers[i]['order_amount']} and drink types are {customers[i]['orders']}")
  for drink in customers[i]['orders']:
    lm = drinks[drink]['rate']
    if order_channel == 'online':
      lm += online_rate_premium
    u = RS_s.rand()
    if anti == 1:
      u = 1-u
    preparing_time = -1/lm*np.log(u)
    # if customers[i]['channel']=='offline':
    rqt = worker.request(priority = priority)
    yield rqt
    yield env.timeout(preparing_time)
    worker.release(rqt)
    if customers[i]['channel']=='offline':
      customers[i]['revenue']+= drinks[drink]['price']
    else: # discount provided to online order/ or the platform(Meituan) need to have a distribution fee.
      customers[i]['revenue']+= drinks[drink]['price'] * online_discount
  if customers[i]['channel']=='offline':  
    outputs['offline_total'] += 1
  else:
    outputs['online_total'] += 1
  customers[i]['finish'] = env.now
  customers[i]['waiting'] = customers[i]['finish']-customers[i]['arrival']
  # linear wating time penalty
  customers[i]['penalized_revenue'] = customers[i]['revenue']*(1-waiting_penalty*customers[i]['waiting'])
  # exponential waiting time penalty
  #customers[i]['penalized_revenue'] = customers[i]['revenue']*np.exp(-waiting_penalty*customers[i]['waiting'])
  if order_channel == 'offline':
    wait_prepare['offline'] -=1
  else:
    wait_prepare['online'] -=1
  # print(f"customer {i} order finished at {env.now:.2f}")

def system_prio(on_off_ratio,RS_s,RS_a,anti):
  env = simpy.Environment()
  customers = {i:{"channel":"online","order_amount":0,"orders":[],"arrival":0,"finish" :0,"revenue":0,'waiting':0,'penalized_revenue':0} for i in range(1,10000)}
  outputs = {'online_total':0,'offline_total':0,'lost_online_customer':0,'lost_offline_customer':0,'lost_online_revenue':0,'lost_offline_revenue':0}
  wait_prepare = {"online":0,"offline":0}
  T = 4  # after 4 hours, stop accept new orders
  worker = simpy.PriorityResource(env,capacity=10)
  env.process(arrival_prio(env,worker,online_arr_rate,offline_arr_rate,customers,outputs,wait_prepare,T,on_off_ratio,RS_s,RS_a,anti))
  env.run()
  total_revenue = sum([customers[i]['revenue'] for i in range(1,10000)])
  total_penlized_revenue = sum([customers[i]['penalized_revenue'] for i in range(1,10000)])
  total_orders = outputs['online_total']+outputs['offline_total']
  average_waiting = sum([customers[i]['waiting'] for i in range(1,10000)])/total_orders
  total_lost_orders = outputs['lost_online_customer'] + outputs['lost_offline_customer']
  total_lost_revenue = outputs['lost_online_revenue']+outputs['lost_offline_revenue']
  #print(f"total revenue is {total_revenue:.2f}")
  return [total_revenue,total_penlized_revenue,average_waiting,total_orders,total_lost_orders,total_lost_revenue]

#Argument: theta, random seed
# this is the threshold we define, below this threshold we follow FIFO, above this threshold we make more offline order
# lower her the ratio, more priority to the offline order

def system_prio_avg(s,on_off_ratio):
    np.random.seed(s)
    RS_s_original=np.random.RandomState(seed=s)
    RS_s_anti=copy.copy(RS_s_original)
    RS_a_original=np.random.RandomState(seed=1000-s)
    RS_a_anti=copy.copy(RS_a_original)
    output1 = system_prio(on_off_ratio,RS_s_original,RS_a_original,0)
    output2 = system_prio(on_off_ratio,RS_s_anti,RS_a_anti,1)
    return [(g+h)/2 for g,h in zip(output1,output2)]


## Simulation and Search for Best Theta

In [ ]:
grid_theta = [0.01,0.1,0.2,0.4,0.6,0.8,1,2,4,6,8,10,100]
theta_dict_revenue = {'mean': {theta:0 for theta in grid_theta},'std': {theta:0 for theta in grid_theta},'CI lower':{theta:0 for theta in grid_theta},'CI upper':{theta:0 for theta in grid_theta},'CI width':{theta:0 for theta in grid_theta} }
theta_dict_penalized_revenue = {'mean': {theta:0 for theta in grid_theta},'std': {theta:0 for theta in grid_theta},'CI lower':{theta:0 for theta in grid_theta},'CI upper':{theta:0 for theta in grid_theta},'CI width':{theta:0 for theta in grid_theta} }
theta_dict_waiting = {'mean': {theta:0 for theta in grid_theta},'std': {theta:0 for theta in grid_theta},'CI lower':{theta:0 for theta in grid_theta},'CI upper':{theta:0 for theta in grid_theta},'CI width':{theta:0 for theta in grid_theta} }
theta_dict_orders = {'mean': {theta:0 for theta in grid_theta},'std': {theta:0 for theta in grid_theta},'CI lower':{theta:0 for theta in grid_theta},'CI upper':{theta:0 for theta in grid_theta},'CI width':{theta:0 for theta in grid_theta} }
theta_dict_lost_orders = {'mean': {theta:0 for theta in grid_theta},'std': {theta:0 for theta in grid_theta},'CI lower':{theta:0 for theta in grid_theta},'CI upper':{theta:0 for theta in grid_theta},'CI width':{theta:0 for theta in grid_theta} }
theta_dict_lost_revenue = {'mean': {theta:0 for theta in grid_theta},'std': {theta:0 for theta in grid_theta},'CI lower':{theta:0 for theta in grid_theta},'CI upper':{theta:0 for theta in grid_theta},'CI width':{theta:0 for theta in grid_theta} }

for theta in grid_theta:
  result = [system_prio_avg(i,theta) for i in range(N_search)]
  result = np.array(result)
  total_revenue = result[:,0]
  total_penlized_revenue = result[:,1]
  average_waiting =  result[:,2]
  total_orders = result[:,3]
  total_lost_orders = result[:,4]
  total_lost_revenue = result[:,5]

  # record total revenue
  mean=np.mean(total_revenue)
  std=np.std(total_revenue,ddof=1)
  lower=mean-spst.norm.ppf(0.975)*std/N**0.5
  upper=mean+spst.norm.ppf(0.975)*std/N**0.5
  width = upper-lower
  theta_dict_revenue['mean'][theta]=mean
  theta_dict_revenue['std'][theta]=std
  theta_dict_revenue['CI lower'][theta]=lower
  theta_dict_revenue['CI upper'][theta]=upper
  theta_dict_revenue['CI width'][theta]=width

  # record penalized revenue
  mean=np.mean(total_penlized_revenue)
  std=np.std(total_penlized_revenue,ddof=1)
  lower=mean-spst.norm.ppf(0.975)*std/N**0.5
  upper=mean+spst.norm.ppf(0.975)*std/N**0.5
  width = upper-lower
  theta_dict_penalized_revenue['mean'][theta]=mean
  theta_dict_penalized_revenue['std'][theta]=std
  theta_dict_penalized_revenue['CI lower'][theta]=lower
  theta_dict_penalized_revenue['CI upper'][theta]=upper
  theta_dict_penalized_revenue['CI width'][theta]=width

  # record waiting time
  mean=np.mean(average_waiting)
  std=np.std(average_waiting,ddof=1)
  lower=mean-spst.norm.ppf(0.975)*std/N**0.5
  upper=mean+spst.norm.ppf(0.975)*std/N**0.5
  width = upper-lower
  theta_dict_waiting['mean'][theta]=mean
  theta_dict_waiting['std'][theta]=std
  theta_dict_waiting['CI lower'][theta]=lower
  theta_dict_waiting['CI upper'][theta]=upper
  theta_dict_waiting['CI width'][theta]=width

  # record total orders
  mean=np.mean(total_orders)
  std=np.std(total_orders,ddof=1)
  lower=mean-spst.norm.ppf(0.975)*std/N**0.5
  upper=mean+spst.norm.ppf(0.975)*std/N**0.5
  width = upper-lower
  theta_dict_orders['mean'][theta]=mean
  theta_dict_orders['std'][theta]=std
  theta_dict_orders['CI lower'][theta]=lower
  theta_dict_orders['CI upper'][theta]=upper
  theta_dict_orders['CI width'][theta]=width

  # record lost orders
  mean=np.mean(total_lost_orders)
  std=np.std(total_lost_orders,ddof=1)
  lower=mean-spst.norm.ppf(0.975)*std/N**0.5
  upper=mean+spst.norm.ppf(0.975)*std/N**0.5
  width = upper-lower
  theta_dict_lost_orders['mean'][theta]=mean
  theta_dict_lost_orders['std'][theta]=std
  theta_dict_lost_orders['CI lower'][theta]=lower
  theta_dict_lost_orders['CI upper'][theta]=upper
  theta_dict_lost_orders['CI width'][theta]=width
 
  # record lost revenue
  mean=np.mean(total_lost_revenue)
  std=np.std(total_lost_revenue,ddof=1)
  lower=mean-spst.norm.ppf(0.975)*std/N**0.5
  upper=mean+spst.norm.ppf(0.975)*std/N**0.5
  width = upper-lower
  theta_dict_lost_revenue['mean'][theta]=mean
  theta_dict_lost_revenue['std'][theta]=std
  theta_dict_lost_revenue['CI lower'][theta]=lower
  theta_dict_lost_revenue['CI upper'][theta]=upper
  theta_dict_lost_revenue['CI width'][theta]=width

  print(f'Finish simulation for theta = {theta}')


Finish simulation for theta = 0.01
Finish simulation for theta = 0.1
Finish simulation for theta = 0.2
Finish simulation for theta = 0.4
Finish simulation for theta = 0.6
Finish simulation for theta = 0.8
Finish simulation for theta = 1
Finish simulation for theta = 2
Finish simulation for theta = 4
Finish simulation for theta = 6
Finish simulation for theta = 8
Finish simulation for theta = 10
Finish simulation for theta = 100


## Performance

### Total Revenue

In [ ]:
get_CI(total_revenue_fifo,"total_revenue_fifo")
pd.DataFrame(theta_dict_revenue)

[total_revenue_fifo] Mean: 27437.976, Std: 303.641, 95% CI: [27378.463, 27497.488], Width: 119.025, N: 100


,mean,std,CI lower,CI upper,CI width
0.01,29322.15250,309.813808,29261.430109,29382.874891,121.444781
0.10,29322.15250,309.813808,29261.430109,29382.874891,121.444781
0.20,29317.19300,309.248564,29256.581395,29377.804605,121.223210
0.40,29306.71500,310.434917,29245.870874,29367.559126,121.688251
0.60,29316.10125,288.966856,29259.464787,29372.737713,113.272926
0.80,29333.39825,315.005214,29271.658363,29395.138137,123.479775
1.00,29362.25375,362.798001,29291.146648,29433.360852,142.214203
2.00,28175.25800,310.208941,28114.458165,28236.057835,121.599670
4.00,27426.88400,301.860316,27367.720465,27486.047535,118.327070
6.00,27437.89325,303.496314,27378.409066,27497.377434,118.968369


### Total Penalized Revenue

In [ ]:
get_CI(total_penlized_revenue_fifo,"total_penlized_revenue_fifo")
pd.DataFrame(theta_dict_penalized_revenue)

[total_penlized_revenue_fifo] Mean: 22304.576, Std: 378.075, 95% CI: [22230.475, 22378.677], Width: 148.203, N: 100


,mean,std,CI lower,CI upper,CI width
0.01,24621.871369,351.732750,24552.933016,24690.809721,137.876704
0.10,24621.871369,351.732750,24552.933016,24690.809721,137.876704
0.20,24616.042182,347.990824,24547.837234,24684.247131,136.409896
0.40,24607.488475,346.878893,24539.501462,24675.475489,135.974028
0.60,24611.703159,336.185777,24545.811958,24677.594361,131.782403
0.80,24575.206165,352.561410,24506.105398,24644.306932,138.201533
1.00,24368.090790,418.453911,24286.075331,24450.106249,164.030919
2.00,23038.705192,369.173157,22966.348583,23111.061801,144.713218
4.00,22296.920571,372.437387,22223.924184,22369.916957,145.992773
6.00,22304.953857,373.755774,22231.699071,22378.208642,146.509571


### Average Waiting Time

In [ ]:
get_CI(average_waiting_fifo,"average_waiting_fifo")
pd.DataFrame(theta_dict_waiting)

[average_waiting_fifo] Mean: 0.757, Std: 0.025, 95% CI: [0.752, 0.762], Width: 0.010, N: 100


,mean,std,CI lower,CI upper,CI width
0.01,0.694109,0.017902,0.690601,0.697618,0.007017
0.10,0.694109,0.017902,0.690601,0.697618,0.007017
0.20,0.694309,0.017670,0.690845,0.697772,0.006926
0.40,0.694509,0.017795,0.691021,0.697996,0.006975
0.60,0.694816,0.018029,0.691282,0.698349,0.007067
0.80,0.701371,0.021075,0.697240,0.705501,0.008261
1.00,0.727906,0.024404,0.723123,0.732689,0.009566
2.00,0.750879,0.023162,0.746340,0.755419,0.009079
4.00,0.756950,0.024934,0.752063,0.761837,0.009774
6.00,0.756924,0.024845,0.752055,0.761794,0.009739


### Total Orders

In [ ]:
get_CI(total_orders_fifo,"total_orders_fifo")
pd.DataFrame(theta_dict_orders)

[total_orders_fifo] Mean: 1082.270, Std: 16.587, 95% CI: [1079.019, 1085.521], Width: 6.502, N: 100


,mean,std,CI lower,CI upper,CI width
0.01,1111.395,16.216854,1108.216555,1114.573445,6.356890
0.10,1111.395,16.216854,1108.216555,1114.573445,6.356890
0.20,1111.120,16.089259,1107.966563,1114.273437,6.306873
0.40,1110.730,16.115495,1107.571421,1113.888579,6.317158
0.60,1111.125,15.597425,1108.067961,1114.182039,6.114078
0.80,1112.675,15.039909,1109.727232,1115.622768,5.895536
1.00,1117.125,17.060702,1113.781164,1120.468836,6.687672
2.00,1096.920,16.128543,1093.758864,1100.081136,6.322273
4.00,1082.090,16.125984,1078.929365,1085.250635,6.321270
6.00,1082.365,16.424122,1079.145931,1085.584069,6.438138


### Total Lost Order

In [ ]:
get_CI(total_lost_orders_fifo,"total_lost_orders_fifo")
pd.DataFrame(theta_dict_lost_orders)

[total_lost_orders_fifo] Mean: 356.755, Std: 23.215, 95% CI: [352.205, 361.305], Width: 9.100, N: 100


,mean,std,CI lower,CI upper,CI width
0.01,327.630,21.636998,323.389226,331.870774,8.481547
0.10,327.630,21.636998,323.389226,331.870774,8.481547
0.20,327.905,21.614704,323.668596,332.141404,8.472808
0.40,328.295,21.699753,324.041927,332.548073,8.506147
0.60,327.900,21.735439,323.639932,332.160068,8.520135
0.80,326.350,20.756464,322.281808,330.418192,8.136384
1.00,321.900,21.411198,317.703482,326.096518,8.393035
2.00,342.105,22.433211,337.708171,346.501829,8.793657
4.00,356.935,22.730265,352.479950,361.390050,8.910100
6.00,356.660,22.744727,352.202115,361.117885,8.915769


### Total Lost Revenue

In [ ]:
get_CI(total_lost_revenue_fifo,"total_lost_revenue_fifo")
pd.DataFrame(theta_dict_lost_revenue)

[total_lost_revenue_fifo] Mean: 10146.479, Std: 761.495, 95% CI: [9997.228, 10295.729], Width: 298.501, N: 100


,mean,std,CI lower,CI upper,CI width
0.01,11010.7260,775.912337,10858.649977,11162.802023,304.152047
0.10,11010.7260,775.912337,10858.649977,11162.802023,304.152047
0.20,11017.5930,775.906720,10865.518077,11169.667923,304.149845
0.40,11032.1010,778.426525,10879.532205,11184.669795,305.137591
0.60,11018.1015,771.828434,10866.825907,11169.377093,302.551186
0.80,10970.4735,742.877703,10824.872146,11116.074854,291.202709
1.00,10651.1850,761.403881,10501.952582,10800.417418,298.464837
2.00,10258.3080,743.481063,10112.588389,10404.027611,291.439221
4.00,10160.9325,758.776696,10012.215000,10309.650000,297.434999
6.00,10147.7340,756.855551,9999.393038,10296.074962,296.681924
